In [2]:
!pip install geocoder
!pip install folium
!pip install geopy
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
!pip install bs4

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print("Libraries imported.")

     |████████████████████████████████| 102kB 4.7MB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
     |████████████████████████████████| 829kB 26.6MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
     |████████████████████████████████| 112kB 6.1MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Libraries imported.


In [18]:
url= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extract_data = requests.get(url).text
wiki_data = BeautifulSoup(extract_data, 'html')

In [19]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto_df = pd.DataFrame(columns = column_names)

content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto_df= toronto_df.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [20]:
toronto_df = toronto_df[toronto_df.Borough!='Not assigned']
toronto_df = toronto_df[toronto_df.Borough!= 0]
toronto_df.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto_df.shape[0]):
    if toronto_df.iloc[i][2] == 'Not assigned':
        toronto_df.iloc[i][2] = toronto_df.iloc[i][1]
        i = i+1

In [26]:
df = toronto_df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.tail()

,Postalcode,Borough,Neighborhood
175,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."
176,M9W\n,Etobicoke\n,"Northwest, West Humber - Clairville"
177,M9X\n,Not assigned\n,Not assigned\n
178,M9Y\n,Not assigned\n,Not assigned\n
179,M9Z\n,Not assigned\n,Not assigned\n


In [27]:

df = df.dropna()
empty = 'Not assigned\n'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]
df.isnull().sum()

Postalcode      0
Borough         0
Neighborhood    0
dtype: int64

In [28]:
df.tail()

,Postalcode,Borough,Neighborhood
170,M9N\n,York\n,Weston
171,M9P\n,Etobicoke\n,Westmount
172,M9R\n,Etobicoke\n,"Kingsview Village, St. Phillips, Martin Grove ..."
175,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."
176,M9W\n,Etobicoke\n,"Northwest, West Humber - Clairville"


In [69]:
def get_latilong(postal_code):    
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.70909000000006, -79.36409999999995]

In [70]:
postal_codes = df['Postalcode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

In [71]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [72]:

df[df.Postalcode == 'M6H\n']


,Postalcode,Borough,Neighborhood,Latitude,Longitude
105,M6H\n,West Toronto\n,"Dufferin, Dovercourt Village",NaN,NaN


In [67]:
address = 'New York, NY'

geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The Geograpical Co-ordinate of New York, NY are {}, {}.'.format(latitude_x, longitude_y))

The Geograpical Co-ordinate of New York, NY are 40.7127281, -74.0060152.


In [68]:
map_ny = folium.Map(location=[latitude_x, longitude_y], zoom_start=10)

for lat, lng, nei in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

ValueError: Location values cannot contain NaNs, got:
[nan, nan]

In [59]:
CLIENT_ID = 'NRUS5AKEOQCTFUXURRMAUHDHUVLAVGMOUGCAD3ULRVEDN5NG' # my Foursquare ID
CLIENT_SECRET = 'PHPUMX4JNATCCF2AXTGOZBHKQCMOFZ3OCVWIBA0J50UEAKLE' # my Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+CLIENT_SECRET)

Your credentails:
CLIENT_ID: NRUS5AKEOQCTFUXURRMAUHDHUVLAVGMOUGCAD3ULRVEDN5NG
CLIENT_SECRET: PHPUMX4JNATCCF2AXTGOZBHKQCMOFZ3OCVWIBA0J50UEAKLE


In [73]:
radius = 500 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_n1, 
   longitude_n1, 
    radius, 
   LIMIT)
results = requests.get(url).json()

NameError: name 'latitude_n1' is not defined

In [76]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7151482 -74.0156573


In [81]:
search_query = 'Iyalian'
radius = 500
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

Iyalian .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=NRUS5AKEOQCTFUXURRMAUHDHUVLAVGMOUGCAD3ULRVEDN5NG&client_secret=PHPUMX4JNATCCF2AXTGOZBHKQCMOFZ3OCVWIBA0J50UEAKLE&ll=40.7151482,-74.0156573&v=20180604&query=Iyalian&radius=500&limit=100'

In [83]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0ef3ae2c789450af196f73'},
 'response': {'venues': []}}

In [84]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

""


KeyError: 'tips'